In [ ]:
# Limpieza de comentarios
import pandas as pd
import re

def clean_comment(comment):
    if pd.isnull(comment):  # Si es NaN, devolvemos string vacío
        return ''
    # Eliminar URLs
    comment = re.sub(r'(http[s]?://\S+|www\.\S+)', '', comment)
    # Emoticones comunes
    comment = re.sub(r'[:;=][-~]?[)D]', '', comment) 
    # Eliminar caracteres no alfabéticos o números
    comment = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\s]', '', comment)
    # Eliminar palabras demasiado cortas (como "a", "b", "c")
    comment = re.sub(r'\b\w{1,2}\b', '', comment)
    # Eliminar espacios múltiples
    comment = re.sub(r'\s+', ' ', comment).strip()
    return comment

def clean_comments(csv_path):
    # Cargar los datos de comentarios
    df = pd.read_csv(csv_path)
    
    # Limpiar los comentarios
    df['cleaned_comment'] = df['text'].apply(clean_comment)
    
    # Eliminar las filas donde el comentario está vacío después de la limpieza
    # También se eliminan los replies a comentarios vacíos.
    print("Cantidad de mensajes iniciales: ", df.shape[0])
    mask_empty = (df['cleaned_comment'].str.strip() == '')
    print("Comentarios vacíos: ", mask_empty.sum())
    video_ids_to_filter = df[mask_empty]["comment_id"]
    mask_replies = ((df['is_reply'] == True) & (df['reply_to_comment_id'].isin(video_ids_to_filter)))
    print("Replies a comentarios vacíos: ", mask_replies.sum())
    mask_to_remove = mask_empty | mask_replies
    df = df[~mask_to_remove] #Eliminación de mensajes vacíos o replies a mensajes vacíos
    print("Cantidad de mensajes finales: ", df.shape[0])
    
    # Guardar los resultados limpios en un nuevo archivo CSV
    #df.to_csv('../data/cleaned_comments.csv', index=False)
    print("Los comentarios han sido limpiados y guardados en '../data/cleaned_comments.csv'.")
    return df

cleaned_comments = clean_comments('../data/comments.csv')

Cantidad de mensajes iniciales:  73147
Comentarios vacíos:  1252
Replies a comentarios vacíos:  26
Cantidad de mensajes finales:  71870
Los comentarios han sido limpiados y guardados en '../data/cleaned_comments.csv'.


In [4]:
# Estadística del número de comentarios por usuario y el número de videos en los que han comentado
def group_comments_by_user(csv_path):
    # Cargar los datos de comentarios
    df = pd.read_csv(csv_path)
    
    # Agrupar por el nombre del autor y contar los comentarios por cada usuario
    user_comment_count = df['author_name'].value_counts().reset_index()
    user_comment_count.columns = ['author_name', 'comment_count']
    
    # Contar en cuántos videos distintos ha comentado cada usuario
    user_video_count = df.groupby('author_name')['video_id'].nunique().reset_index()
    user_video_count.columns = ['author_name', 'video_count']
    
    # Fusionar ambas tablas (comentarios y videos)
    user_data = pd.merge(user_comment_count, user_video_count, on='author_name')
    
    # Ordenar los usuarios por la cantidad de comentarios
    user_data = user_data.sort_values(by='comment_count', ascending=False)
    
    return user_data

def save_grouped_comments(user_data, output_path='../data/user_comment_video_counts.csv'):
    # Guardar los resultados en un CSV
    user_data.to_csv(output_path, index=False)
    print(f"Datos de comentarios y videos por usuario guardados en: {output_path}")

user_data = group_comments_by_user("../data/cleaned_comments.csv")
# Guardar el resultado en un CSV
save_grouped_comments(user_data)

Datos de comentarios y videos por usuario guardados en: ../data/user_comment_video_counts.csv
